In [1]:
import spacy
import xml.etree.ElementTree as ET
from spacy.symbols import nsubj, dobj, pobj, iobj, neg, xcomp, VERB
import pandas as pd
import re
import os

nlp=spacy.load('en_core_web_lg')
 

In [ ]:
[{'head': 'us', 'type': 'provide aid', 'tail': 'ukraine'}] [{'head': 'us', 'type': 'provide aid', 'tail': 'the pentagon'}, {'head': 'ukraine', 'type': 'provide aid', 'tail': 'military aid'}]
[{'head': 'russian russian russian russian military military', 'type': 'consult', 'tail': 'a russian military'}] [{'head': 'russian retreat', 'type': 'consult', 'tail': "a'military failure'but"}]
[{'head': 'which', 'type': 'make public statement', 'tail': 'the'}] [{'head': 'the palacio san martin', 'type': 'make public statement', 'tail': 'france'}, {'head': 'valuable information', 'type': 'coerce', 'tail': 'the framework'}]
[{'head': '<MASK>', 'type': 'express intend to cooperate', 'tail': 'the self self selfies'}] [{'head': 'the apparent first ukrainian troops', 'type': 'express intend to cooperate', 'tail': 'snapped selfies'}]
[{'head': 'which', 'type': 'yield', 'tail': 'which'}] [{'head': 'which', 'type': 'yield', 'tail': 'many younger pakistanis'}]
[{'head': 'cristina fernandez fernandez fernandez', 'type': 'disapprove', 'tail': 'the president cristina fernandez'}] [{'head': 'vice - president cristina fernandez de kirchner', 'type': 'disapprove', 'tail': 'the judge'}]

In [1]:
ex1 = [{'head': 'us', 'type': 'provide aid', 'tail': 'ukraine'}] 
ex1_g = [{'head': 'us', 'type': 'provide aid', 'tail': 'the pentagon'}, {'head': 'ukraine', 'type': 'provide aid', 'tail': 'military aid'}]

In [6]:
if "provide aid" in ex1[0]["type"]:
    print("hellya")

In [ ]:
[{'head': 'us', 'type': 'provide aid', 'tail': 'ukraine'}] [{'head': 'us', 'type': 'provide aid', 'tail': 'the pentagon'}, {'head': 'ukraine', 'type': 'provide aid', 'tail': 'military aid'}]

In [3]:
#for pretrain data

data = pd.read_csv("data/out_data/articles_url_coref.csv.done_1.csv", index_col = 0)

In [5]:
data.head()

,text,label
0,For more on our coverage of the war in Ukraine...,<triplet> loaded ships <subj> Ukraine <obj> Co...
1,"— UN grain coordinator November 1, 2022 10:09p...",<triplet> UN grain coordinator <subj> Ukraine ...
2,"""The grain corridor needs reliable and long-te...",<triplet> The grain corridor <subj> reliable a...
3,"""Russia must clearly understand that Russia wi...",<triplet> that <subj> Russia's food exports <o...
4,"7:20pm: Russian attacks on Ukraine's water, en...",<triplet> Russia <subj> a huge volley <obj> Fi...


In [56]:
data["text2"] =data.text.apply(lambda x: x.replace("\xa0", " "))

In [83]:
#idx 25 is actually a good example

In [159]:
data.label.iloc[25]

'<triplet> Pakistan prime minister Shehbaz Sharif <subj> China <obj> Consult'

In [160]:
data.text2.iloc[25]

"Pakistan prime minister Shehbaz Sharif is visiting China at the invitation of Pakistan prime minister Shehbaz Sharif's Chinese counterpart Li Keqiang."

In [161]:
nlp.get_pipe("ner").labels
#we care about GPE, NORP, EVENTS, FAC, LAW, ORG, PERSON

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [162]:
sentence = data.text.iloc[25]

In [163]:
doc = nlp(sentence)

In [164]:
from spacy import displacy
svg = displacy.render(nlp(sentence),style = "dep", jupyter=True, options = {"distance": 90}, minify = False)

In [165]:
for token in doc.ents:
    print(token, token.label_)

Pakistan GPE
Shehbaz Sharif PERSON
China GPE
Pakistan GPE
Shehbaz Sharif PERSON
Chinese NORP
Li Keqiang PERSON


In [23]:
doc = nlp(sentence)
verbs = []
dict = {}

for possible_verb in doc:
    if possible_verb.pos == VERB:
        if neg in [child.dep for child in possible_verb.children]: continue
        else: 
            for possible_subject in possible_verb.children: 
                if possible_subject.dep == xcomp:   #subj / obj of composed verb should also be subj / obj of main verb
                    main_verb = possible_subject
                    main_idx = possible_subject.idx
                    for token in doc.ents:
                        if token.label_ in ["GPE", "NORP", "EVENTS", "FAC", "LAW", "ORG", "PERSON"]:
                            if token.root.dep_ == "poss":
                                if token.root.head.head.idx == possible_verb.idx:
                                    verbs.append([main_idx, main_verb.lemma_, token.text, token.root.head.dep_])
                                    if main_idx in dict.keys(): dict[main_idx] += 1
                                    else: dict[main_idx] = 1
                            else:
                                if token.root.head.idx == possible_verb.idx:
                                    verbs.append([possible_verb.idx, possible_verb.lemma_, token.text, token.root.dep_])
                                    if possible_verb in dict.keys(): dict[possible_verb] += 1
                                    else: dict[possible_verb] = 1

            for token in doc.ents:
                if token.label_ in ["GPE", "NORP", "EVENTS", "FAC", "LAW", "ORG", "PERSON"]:
                    if token.root.dep_ == "poss":
                        if token.root.head.head.idx == possible_verb.idx:
                            verbs.append([possible_verb.idx, possible_verb.lemma_, token.text, token.root.head.dep_])
                            if possible_verb in dict.keys(): dict[possible_verb] += 1
                            else: dict[possible_verb] = 1
                    else:
                        if token.root.head.idx == possible_verb.idx:
                            verbs.append([possible_verb.idx, possible_verb.lemma_, token.text, token.root.dep_])
                            if possible_verb in dict.keys(): dict[possible_verb] += 1
                            else: dict[possible_verb] = 1

trip_idx = [key for key in dict if dict[key] > 1]

In [24]:
verbs

[[16, 'run', 'Donald Trump', 'nsubj'],
 [77, 'call', 'Donald Trump', 'nsubj'],
 [77, 'call', 'Donald Trump', 'nsubj'],
 [107, 'storm', 'Capitol', 'dobj']]

In [168]:
dict

{visiting: 2}

In [32]:
#for pretrain data

data = pd.read_csv("data/out_data/articles_url_coref.csv.done.csv", index_col = 0)
data["len"] = data.text.apply(lambda x: len(x))

In [33]:
import numpy as np
np.percentile(data.len.to_list(),75)

561.0

In [46]:
smol = data[data.len < 400]

In [47]:
data.shape, smol.shape

((565, 4), (360, 4))

In [28]:
sentence = smol.text.iloc[43]

In [20]:
from spacy import displacy
svg = displacy.render(nlp(smol.text.iloc[43]),style = "dep", jupyter=True, options = {"distance": 90}, minify = False)

In [35]:
smol.label.to_list()

['<triplet> Benjamin Netanyahu <subj> Yair Lapid <obj> Coerce',
 '<triplet> Shehbaz Sharif <subj> China <obj> Consult',
 '<triplet> China <subj> Shehbaz Sharif <obj> Engage In Diplomatic Cooperation',
 '<triplet> Dawn <subj> Shehbaz Sharif <obj> Make Public Statement',
 '<triplet> Shehbaz Sharif <subj> Confucius <obj> Make Public Statement',
 '<triplet> Russia <subj> Ukraine <obj> Fight',
 '<triplet> US <subj> US <obj> Appeal',
 '<triplet> Russia <subj> Russia <obj> Engage In Material Cooperation',
 '<triplet> Brazil <subj> Luíz Inácio Lula da Silva <obj> Make Public Statement',
 '<triplet> the Associated Press <subj> Iran <obj> Make Public Statement',
 '<triplet> Oleg Tinkov <subj> Tinkoff Bank <obj> Engage In Material Cooperation',
 '<triplet> Oleg Tinkov <subj> - TCS Group Holding <obj> Engage In Material Cooperation',
 "<triplet> Dorsey <subj> Elon Musk's <obj> Engage In Diplomatic Cooperation",
 "<triplet> Elon Musk <subj> Elon Musk's <obj> Engage In Material Cooperation",
 '<trip

In [36]:
smol

,text,label,len
2,Polls say the race is too close to predict.Aft...,<triplet> Benjamin Netanyahu <subj> Yair Lapid...,245
4,Pakistan prime minister Shehbaz Sharif is visi...,<triplet> Shehbaz Sharif <subj> China <obj> Co...,150
5,Chinese foreign ministry spokesperson Zhao Lij...,<triplet> China <subj> Shehbaz Sharif <obj> En...,237
6,"Separately, Dawn reported Pakistan prime minis...",<triplet> Dawn <subj> Shehbaz Sharif <obj> Mak...,323
7,“There are three types of good friends: one th...,<triplet> Shehbaz Sharif <subj> Confucius <obj...,336
...,...,...,...
559,Barack Obama warned Democrats on Saturday that...,<triplet> Barack Obama <subj> Democrats <obj> ...,177
561,the Republican and two Democrats all urged Ame...,<triplet> Republican <subj> Republican <obj> A...,78
562,As it happened: Pennsylvania ralliesFive reaso...,<triplet> Donald Trump <subj> Pennsylvanians <...,372
563,BEIJING: China on Sunday reported China's high...,<triplet> China <subj> China <obj> Make Public...,271


In [38]:
data["trips"] = data.label.apply(lambda x: x.count("<triplet>"))

In [40]:
data.groupby("trips").text.count()

trips
1    531
2     23
3     11
Name: text, dtype: int64

In [42]:
smol.groupby("trips").text.count()

trips
1    353
2      7
Name: text, dtype: int64

In [233]:
def read_lines(inputparsed):    
    """takes input from CoreNLP sentence parsed file and returns sentences"""
    #parse all lines from CoreNLP sentence split
    parsed = open(inputparsed, encoding = "utf-8")
    parsedfile = parsed.readlines()
    parsedlines = []

    #Only keep those lines which have Sentence #n in the line before
    for idx, text in enumerate(parsedfile):
        if text.startswith("Sentence #"):
            parsedlines.append(parsedfile[idx+1].replace('\n','').strip())
    
    return parsedlines

pico_path = r"C:/Users/svawe/Thesis_RelationExtraction_PoliticsNews/soft_data/src/add_labels/dictionaries/PETR.Internal.Coding.Ontology.txt"
verb_path = r"C:/Users/svawe/Thesis_RelationExtraction_PoliticsNews/soft_data/src/add_labels/dictionaries/newdict.txt"
# def verb_code_dict(pico_path, verb_path):
#     """reads coding ontology and verb lists, 
#     directly matches verbs to their CAMEO codes and returns this verbs:codes dictionairy.
#     verb with codes that cannot be read are printed out as full line of the file"""
#read PETRARCH Internal Coding Ontology (= pico)
pico_path = os.path.join(os.getcwd(), pico_path)
pico_file = open(pico_path, 'r')
pico_lines = pico_file.readlines()

#get all 20 codes with their respective code
main_codes = {}                             #we run one iteration for all the main codes, only main codes contain relation name
for line in pico_lines:
    line = line.split('#')
    if line[0] == "" or line[0] == "\n":    #only intro comments and empty lines
        continue
    else: 
        code_split = line[0].split(":")     #splits into CAMEO code and related hex
        if len(line) > 1 and code_split[0][2] == "0":      #only main categories have 0 in 3rd idx, [cat_num 0] -> [010]
            main_codes[code_split[0][:2]] = line[-1].replace("\n","")

#map code to code we want to use in the training
map_codes = {"DiplomaticCoop" : "Engage In Diplomatic Cooperation", 
            "MaterialCoop" : "Engage In Material Cooperation",
            "ProvideAid" : "Provide Aid",
            "Exhibit Force Posture": "Exhibit Military Posture",
            "Use Unconventional Mass Violence" : "Engage In Unconventional Mass Violence"}
main_codes = {k: (map_codes[v] if v in map_codes else v) for k, v in main_codes.items()}

#read verbs and match their code to the relation extracted in main_codes
verb_path = os.path.join(os.getcwd(), verb_path)
verb_file = open(verb_path, 'r')
verb_lines = verb_file.readlines()

verb_dict = {}
for line in verb_lines:
    if line[0] == "#":
        continue
    elif line.startswith("---"):    #main verbs have a lead code, which is applied to all very in the section
                                    #unless a separate code is specified for a specific verb in section
        try: cur_main_code = re.split("\[|\]|---", line)[2].replace(":","")[:2]  #we only need main codes which are first two numbers
                                                                            #sometimes code starts with ":", e.g.: ---  OFFEND   [:110]  ---
                                                                            #we just remove those to get the main code
        except:                     #depending on chosen verb dictionairy, there may be main verbs without lead codes
            print("couldn't finde code in: ", line.replace("\n","")) 
            cur_main_code == "--"
        if cur_main_code == "": cur_main_code = "--"
    elif line == "\n":              #skip empty lines
        continue
    elif line[0] == "-" or line[0] == "~" or line[0] == "+" or line[0] == "&": #removes all special structures we cannot use
        continue
    else:
        if len(re.split("\[|\]", line)) > 1:    #verbs with their own code, e.g.: AFFIRM [051] 
            code = re.split("\[|\]", line)[1].replace(":","")[:2]
            if code != "--":
                if "{" in line:         #conjugated verbs, e.g. "APPLY {APPLYING APPLIED APPLIES } [020]"
                    line_s = re.split("\{|\}", line)    #split at { and }
                    verb_dict[line_s[0].lower()] = main_codes[code] 
                    for word in line_s[1].split():
                        verb_dict[word.lower()] = main_codes[code]
                else:
                    word = re.split("\[|\]", line)[0]
                    verb_dict[word.lower()] = main_codes[code]
        else:
            if cur_main_code != "--":
                if "{" in line:         #e.g. "HURRY {HURRIES HURRYING HURRIED }" 
                    line_s = re.split("\{|\}", line)    #split at { and }
                    verb_dict[line_s[0].lower()] = main_codes[cur_main_code]
                    for word in line_s[1].split():
                        verb_dict[word.lower()] = main_codes[cur_main_code]
                else:                   #only single words with sometimes comments, e.g.: CENSURE  # JON 5/17/95
                    word = line.split("#")[0].rstrip()    #gets part before "#", removes all whitespaces to the right
                    verb_dict[word.lower()] = main_codes[cur_main_code]


couldn't finde code in:  --- DEFEND  ###
couldn't finde code in:  --- REVOKE_   ###
couldn't finde code in:  --- SEND   ###
couldn't finde code in:  --- COLLAPSE  ###


In [51]:
verb_dict

{'abandon': 'Yield',
 'explain': 'Make Public Statement',
 'comment': 'Make Public Statement',
 'assert': 'Make Public Statement',
 'SAY  ': 'Make Public Statement',
 'says': 'Make Public Statement',
 'said': 'Make Public Statement',
 'saying': 'Make Public Statement',
 'CLARIFY ': 'Make Public Statement',
 'clarifies': 'Make Public Statement',
 'clarified': 'Make Public Statement',
 'clarifying': 'Make Public Statement',
 'elucidate': 'Make Public Statement',
 'explicate': 'Make Public Statement',
 'rationalize': 'Make Public Statement',
 'express ': 'Make Public Statement',
 'articulate': 'Make Public Statement',
 'state': 'Make Public Statement',
 'TELL ': 'Make Public Statement',
 'tells': 'Make Public Statement',
 'told': 'Make Public Statement',
 'telling': 'Make Public Statement',
 'narrate': 'Make Public Statement',
 'describe': 'Make Public Statement',
 'define': 'Make Public Statement',
 'depict': 'Make Public Statement',
 'detail': 'Make Public Statement',
 'expound': 'Make 

In [189]:
#placeholder dict

verb_path = os.path.join(os.getcwd(), verb_path)
verb_file = open(verb_path, 'r')
verb_lines = verb_file.readlines()

pre_dict = {}
filter_list = []
for line in verb_lines:
    if line.startswith("&"):
        cur_filter = line.rstrip()
    elif line.startswith("\n"):
        pre_dict[cur_filter.lower()] = filter_list
        cur_filter = ""
        filter_list = []
    elif line.startswith("+") and cur_filter != "":
        filter_list.append(line.rstrip()[1:].replace("_", "").lower())
del pre_dict[""]

In [249]:
def gen_poss(line, verb_match):
    poss = []

    #replace special tokens in text that are clear at this point
    line = line.replace("*", verb_match)
    line = line.replace("- ", "")
    line = line.replace("+","")
    line = line.replace("%","")
    line = line.replace("^","")
    line = line.replace("$","")

    #split line by possibility indicators and code (always ends possibility)
    #example.: "- $ * (P ON KILLING (P OF + [010] #  COMMENT <ELH 07 May 2008>"
    poss_split = re.split("\(P |\[.*]",line) 

    if len(poss_split) > 2: #2 is if no (P in the line
        #only combining the first (P, as they share the same code 
        #and the longer version will never be contained in a text if the shorter isnt
        poss.append(strip_multiple_whitespaces(" ".join(poss_split[:2])).lower().rstrip().lstrip())
    else: 
        poss.append(strip_multiple_whitespaces(poss_split[0].lower().rstrip().lstrip()))

    cleaned = []
    for text in list(set(poss)):
        c = 0
        for tag in list(pre_dict.keys()):
            if tag in text:
                for replacement in pre_dict[tag]:
                    cleaned.append(text.replace(tag, replacement))
                    c += 1
        if c == 0:
            cleaned.append(text)

    return cleaned
    

In [250]:
#special verb dict

verb_path = os.path.join(os.getcwd(), verb_path)
verb_file = open(verb_path, 'r')
verb_lines = verb_file.readlines()

spec = {}
spec_code = {}

count = 0
for line in verb_lines:
    if line.startswith("- "):
        #get main verb as dict key
        try: 
            verb_match = re.search("# *\w+", line).group()
            verb_match = re.search("\w+", verb_match).group()
            verb_match = verb_match.replace("_", " ").lower()
        except: 
            count += 1
            

        #get code for line
        try:
            code = re.search("\[.*]", line).group()[1:3]
            if code != "--":
                #get all possibility that the line indicates
                poss = gen_poss(line, verb_match)
                for pattern in poss:
                    spec_code[pattern] = main_codes[code]
                spec[verb_match] = poss
        except:
            count += 1

print(f"{count} patterns could not be loaded")



22 patterns could not be loaded


In [253]:
example = "What would happen those who worry that German business is too closely woven into China ask, in the event that China invaded Taiwan?"

In [261]:
for poss_pattern in spec["mention"]:
    for word in poss_pattern.split():
        
    if all(poss_pattern.split() in example):
        print(poss_pattern)

TypeError: 'in <string>' requires string as left operand, not list

In [285]:
if "poss_pattern" in locals():
    print(True)

True


In [293]:
verb_dict["lose"]

KeyError: 'lose'

In [292]:
spec["lose"]

KeyError: 'lose'

In [235]:
len(verb_dict.keys())

1522

In [237]:
len(spec.keys())

798

In [240]:
all_verbs = list(verb_dict.keys())
for key in spec.keys():
    if key not in verb_dict.keys():
        all_verbs.append(key)

In [241]:
len(all_verbs)

1952

In [238]:
for key in spec.keys():
    if key not in verb_dict.keys():
        print(key)

say
express
tell
refer
defer
bid
strike
squared off
fight
shot and
kick
combat
wrestle
duel
gunfire
complain
debate
object
meet
convene
anchor
fell
blew up
frustrate
distress
speak
confer
attend
pulled out
go
went to
leave
en route
continue
advance
oversee
control
monitor
lead
address
decree
stepped up
step up
fuel
name
appoint
elect
designate
vote
decide
determine
opt
pick
endorse
give
contribute
supply
donate
lend
break
wipe out
demolish
wiped out
uphold
keep
reserve
save
maintain
show
present
display
loot
close
complete
stop
abort
quit
end
conclude
halt
called off
wrapped up
closure
reclose
finalize
carried out
carry out
commit
fulfill
relinquish
surrender
concede
yield
evacuate
escape
flee
defect
evade
withdraw
pullback
resign
vacate
release
free
resist
deny
ruled out
reject
refuse
decline
suspect
doubt
find
uncover
take
arrest
arrested and
apprehend
killed and
murder
see
snub
repel
violate
mark
brand
sweep
seek
search
hunt
study
reaffirm
confirm
affirm
confidence
enrage
hold
withh